<a href="https://colab.research.google.com/github/ideablast/NLPer_chatbot/blob/hj/FAQ_%EB%AC%B8%EB%8B%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument
import pandas as pd

In [10]:
wear_data = pd.read_csv("/content/drive/My Drive/clothing.csv")
customer = wear_data[wear_data.SPEAKER == "고객"].SENTENCE
store = wear_data[wear_data.SPEAKER == "점원"].SENTENCE


In [13]:
prev = "고객"
store_arr = []
customer_arr = []
store_stc = ""
customer_stc = ""

for i in range(wear_data.shape[0]):
    if (prev == wear_data.iloc[i].SPEAKER):
        if prev == "점원":
             store_stc += (" "+wear_data.iloc[i].SENTENCE)
        else : 
            customer_stc += (" "+wear_data.iloc[i].SENTENCE)
            
    elif prev == "점원":
        store_arr.append(store_stc)
        customer_stc = wear_data.iloc[i].SENTENCE
        prev = "고객"
    else :
        customer_arr.append(customer_stc)
        store_stc = wear_data.iloc[i].SENTENCE
        prev = "점원"

faqs = []

for i in range(len(store_arr)):
    faqs_tmp =[]
    faqs_tmp.append(str(i+1))
    faqs_tmp.append(customer_arr[i])
    faqs_tmp.append(store_arr[i])

    faqs.append(faqs_tmp)

# print(store_arr[-1])
# print(customer_arr[-1]) # 자료 상에서 이후에는 계속 고객의 물음만 계속된다. 코드 레벨에서 이 부분은 빼게 구현했다. (stc는 만들어지지만 arr에 append 안하게 된다.)

In [14]:
import jpype
from konlpy.tag import Kkma

In [38]:
kkma = Kkma()
filter_kkma = ['NNG', 'NNP','OL','VA','VV','VXV']

def tokenizer_kkma(doc):
    # 꼬꼬마 형태소 분석기가 자바 기반이어서 파이썬에서 자바함수들을 실행할 수 있는 명령어 (jpype) 를 써줘야한다.
    jpype.attachThreadToJVM()       
    token_doc = ["/".join(word) for word in kkma.pos(doc)]
    return token_doc

def tokenize_kkma_noun_verb(doc):
    jpype.attachThreadToJVM()
    token_doc = ["/".join(word) for word in kkma.pos(doc) if word[1] in filter_kkma]
    return token_doc

In [16]:
token_faqs = [(tokenizer_kkma(row[1]), row[0]) for row in faqs]
tagged_faqs = [TaggedDocument(d,[c]) for d,c in token_faqs]

In [17]:
# 모델 만들기
# cpu 몇 개 쓸 건지
import multiprocessing
# 내 컴에 있는 cpu 갯수 cores 에 저장
cores = multiprocessing.cpu_count()
# vector_size : 임베딩할 벡터 차원
# negaive : negative sampling
d2v_faqs = doc2vec.Doc2Vec(
    vector_size = 100,
    alpha = 0.025,
    min_alpha = 0.025,
    hs = 1,
    negative = 0,
    dm = 0,
    dbow_words = 1,
    min_count = 1,
    workers = cores,
    seed = 0
)

# 단어 사전 만들기
d2v_faqs.build_vocab(tagged_faqs)
for epoch in range(4):
  # 모델 학습
  print(epoch)
  d2v_faqs.train(tagged_faqs,
                 total_examples = d2v_faqs.corpus_count,
                 epochs = d2v_faqs.epochs)
  d2v_faqs.alpha -=0.0025
  d2v_faqs.min_alpha = d2v_faqs.min_alpha

0
1
2
3


In [27]:
input_question = input()
token_test = tokenizer_kkma(input_question)
predict_vector = d2v_faqs.infer_vector(token_test)
result = d2v_faqs.docvecs.most_similar([predict_vector],topn=5)
for i in range(5):
            print("{}위. {}, {}, {}".format(i+1, result[i][1], result[i][0], faqs[int(result[i][0])-1][2]))

내구성 좋은 가방은 요즘 어떤게 잘나가요?
1위. 0.7844632863998413, 1290, 조금 달랑달랑 하는 게 잘 나가요 
2위. 0.7626214027404785, 2865, 개인 취향이기 때문에 그거는 정해진 게 없어요 
3위. 0.755354106426239, 268, 다양하게 잘 나가요
4위. 0.7467412948608398, 1300, 알레르기 있으세요?
5위. 0.7464330196380615, 3096, 이런 쪽으로 이것과 크로스 연결된 거 많이 찾으세요


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [41]:
question_list_bag = ['근무하신지 얼마나 되셨어요?','이가게 월세가 얼마에요?','내일 저랑 데이트 하실래요?','여기 화장실이 어디에요?','이 가방 최대 몇 리터까지 들어갈까요?','이거 끈은 분리하고 세탁기에 돌려야 하나요?','이거 방수 되는 거죠?','우리 애가 이번에 초등학교 들어가는데 가방 추천좀 해주세요.','내구성 좋은 가방은 요즘 어떤게 잘나가요?']
question_list_dress = ['이 라이더 자켓 어제 샀는데 영수증 없어도 환불 되나요?','가볍고 따뜻한 패딩조끼 있나요?,물 잘 안빠지는 청바지 있나요?','이 스웨터 여기 살짝 올이 나갔는데 할인 안해주시나요?','가게 앞 마네킹에 피팅된 바지 다른 색깔 있나요?,수선 맡기고 옷 택배로 받아 볼 수 있나요?','지역상품권 결제 가능한가요?','카운터에 있는 점원이 입고 있는 옷 맘에 드는데 어떤 제품인가요?','소개팅에 입고 나갈만한 옷 추천 가능한가요?','이 가게 단골이고 사장님이랑 잘 아는데 현금 결제하면 할인 없나요?']
question_list_acs= ['침은 모두 알러지 방지 침인가요?','로즈골드 색상도 있나요?','귀걸이는 어떻게 관리해야 하나요?','요즘 잘나가는 제품은 뭔가요?,이 목걸이랑 세트인 귀걸이도 있나요?']
question_list_shoes= ['신어보고 구매 가능할까요?','겨울에 신기는 너무 춥죠?','밖에 날씨가 너무 춥네요','신발이 너무 가벼운거같아요 좀 무게감있는건 없나요?','다른브랜드에서는 비슷한게있나요?']

for input_question in question_list_bag:
   token_test = tokenizer_kkma(input_question)
   predict_vector = d2v_faqs.infer_vector(token_test)
   result = d2v_faqs.docvecs.most_similar([predict_vector],topn=5)
   print(input_question)
   for i in range(5):
     print("{}위. {}, {}, {}".format(i+1, result[i][1], result[i][0], faqs[int(result[i][0])-1][2]))
     print("")



근무하신지 얼마나 되셨어요?
1위. 0.6097705960273743, 4598, 현재 20프로 할인하고 있습니다

2위. 0.5646914839744568, 6392, 1400원이에요

3위. 0.5622261762619019, 6853, 그 옷이 십칠만 사백 원이에요

4위. 0.5244528651237488, 3938, 이쪽에서 결제 도와드릴게요

5위. 0.4977331757545471, 3473, 선물포장으로 해드릴까요?

이가게 월세가 얼마에요?
1위. 0.585567831993103, 4944, 20% 할인해서 189400원입니다

2위. 0.5665419101715088, 5348, 네 가능합니다

3위. 0.5664279460906982, 4773, 35만원입니다

4위. 0.5294262170791626, 4373, 그럼요

5위. 0.5069411993026733, 3679, 삼성페이는 결제가 안되세요

내일 저랑 데이트 하실래요?
1위. 0.6055468916893005, 3183, 네, 같이 준비 해 두겠습니다.

2위. 0.5809895992279053, 3324, 물론입니다 내일 편하신 시간에 오시면 준비해놓겠습니다

3위. 0.5439537763595581, 3965, 네 저지는 카운터 오른쪽편에 있습니다

4위. 0.5350192785263062, 4385, 네 여기있습니다

5위. 0.5337527990341187, 1522, 사이즈 몇 신으세요?

여기 화장실이 어디에요?
1위. 0.7114323973655701, 1952, 45는 이쪽에 있어요

2위. 0.6570717096328735, 1001, 여기 밖에 있어요

3위. 0.5846936702728271, 293, 여기하고 바깥쪽에 있어요

4위. 0.5687581300735474, 5318, 저쪽 청바지 코너 옆에 있습니다

5위. 0.5587208271026611, 4639, 네

이 가방 최대 몇 리터까지 들어갈까요?
1위. 0.6282836198806763, 55

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [42]:
for input_question in question_list_dress:
   token_test = tokenizer_kkma(input_question)
   predict_vector = d2v_faqs.infer_vector(token_test)
   result = d2v_faqs.docvecs.most_similar([predict_vector],topn=5)
   print(input_question)
   for i in range(5):
     print("{}위. {}, {}, {}".format(i+1, result[i][1], result[i][0], faqs[int(result[i][0])-1][2]))
     print("")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


이 라이더 자켓 어제 샀는데 영수증 없어도 환불 되나요?
1위. 0.5431357622146606, 5295, 구입하신 블라우스 좀 보여주시겠어요?

2위. 0.5288113951683044, 6723, 네

3위. 0.5242164134979248, 4565, 죄송하지만 이 제품은 환불은 불가능하고 교환은 가능하십니다 고객님

4위. 0.5196164846420288, 3749, 네 그럼요

5위. 0.4963870346546173, 749, 네 가능합니다

가볍고 따뜻한 패딩조끼 있나요?,물 잘 안빠지는 청바지 있나요?
1위. 0.44456756114959717, 380, 가격대는 다양해요

2위. 0.4361600875854492, 7014, 패딩 조끼 사이즈는 110까지 있어요

3위. 0.43015050888061523, 615, 안에 기모가 살짝 들어 있어요

4위. 0.4241315424442291, 175, 내일 들어와요

5위. 0.42337968945503235, 6856, 저렴한 옷은 이런 제품 괜찮아요

이 스웨터 여기 살짝 올이 나갔는데 할인 안해주시나요?
1위. 0.5781936645507812, 5753, 네 바로 바꿔드릴게요

2위. 0.4338272213935852, 3789, 현금결제로 하시면 10프로 할인해드립니다.

3위. 0.4257911443710327, 5072, 저희 앞 쪽에 걸려있는 상품이 현재 가장 인기 있는 제품이에요

4위. 0.4200326204299927, 3631, 이 부분만 교체해드리면 되나요?

5위. 0.4198552072048187, 5098, 네 알겠습니다

가게 앞 마네킹에 피팅된 바지 다른 색깔 있나요?,수선 맡기고 옷 택배로 받아 볼 수 있나요?
1위. 0.4900781810283661, 3801, 지금 전국품절이라 2주 이상 대기하셔야합니다.

2위. 0.468337744474411, 4548, 일주일 후에 가능합니다

3위. 0.43664926290512085, 1554, 저희 센터가 있

In [43]:
for input_question in question_list_acs:
   token_test = tokenizer_kkma(input_question)
   predict_vector = d2v_faqs.infer_vector(token_test)
   result = d2v_faqs.docvecs.most_similar([predict_vector],topn=5)
   print(input_question)
   for i in range(5):
     print("{}위. {}, {}, {}".format(i+1, result[i][1], result[i][0], faqs[int(result[i][0])-1][2]))
     print("")

침은 모두 알러지 방지 침인가요?
1위. 0.7525830864906311, 3292, 네

2위. 0.746813178062439, 3202, 만원짜리들은 다 알러지 방지에요

3위. 0.737785816192627, 3234, 네, 있습니다. 

4위. 0.6926695108413696, 3398, 네 이 제품입니다

5위. 0.6262699961662292, 60, 액세서리예요 금 아니예요

로즈골드 색상도 있나요?
1위. 0.7377068996429443, 1258, 네

2위. 0.6879805326461792, 3269, 네 포장해드릴게요 잠시만 기다려주세요 

3위. 0.6684713959693909, 1379, 바는 없어요. 한 돈에 23만 8천 원 예상하시면 됩니다

4위. 0.5889818668365479, 3534, 네 이쪽에 있습니다

5위. 0.5509798526763916, 1217, 네

귀걸이는 어떻게 관리해야 하나요?
1위. 0.6887160539627075, 444, 세탁소에 세탁 맡기는 게 제일 좋아요

2위. 0.6884642839431763, 4867, 부드러운 천으로 닦아주세요

3위. 0.6535139083862305, 4472, 물세탁 간단하게 해주시면 됩니다

4위. 0.5767284631729126, 4547, 네

5위. 0.5725282430648804, 1228, 별도로 비닐에 담아서 깔끔한 상태로 보관해주는 게 좋아요

요즘 잘나가는 제품은 뭔가요?,이 목걸이랑 세트인 귀걸이도 있나요?
1위. 0.5848991870880127, 3228, 네 고객님  그 귀걸이는 단품만 구매 가능합니다.

2위. 0.5728444457054138, 3661, 백금이에요

3위. 0.5627347230911255, 1290, 조금 달랑달랑 하는 게 잘 나가요 

4위. 0.4981006681919098, 3525, 은 제품은 아니에요

5위. 0.4921590983867645, 142, 앞에만 은인 거도 있고 침까지 은인 거

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [44]:
for input_question in question_list_shoes:
   token_test = tokenizer_kkma(input_question)
   predict_vector = d2v_faqs.infer_vector(token_test)
   result = d2v_faqs.docvecs.most_similar([predict_vector],topn=5)
   print(input_question)
   for i in range(5):
     print("{}위. {}, {}, {}".format(i+1, result[i][1], result[i][0], faqs[int(result[i][0])-1][2]))
     print("")

신어보고 구매 가능할까요?
1위. 0.5756327509880066, 4836, 네 영수증 지참하시면 가능합니다

2위. 0.5537137985229492, 5145, 아니요 품절이에요

3위. 0.5292538404464722, 3184, 네, 가능합니다.

4위. 0.5291168689727783, 5241, 네 가능하세요

5위. 0.5227987766265869, 3617, 네 가능한데 사이즈가 어떻게 되세요?

겨울에 신기는 너무 춥죠?
1위. 0.5846367478370667, 695, 네 그렇긴해요

2위. 0.4799661338329315, 1938, 천이 좀 두껍게 나온건데 그것도 봄 신발이예요

3위. 0.45976895093917847, 578, 인디언 핑크와 조끼 한 번 입어보세요

4위. 0.4581145942211151, 587, 원피스예요

5위. 0.45733484625816345, 6371, 그거는 원래 9만 9천원인데 세일해서 6만원이에요

밖에 날씨가 너무 춥네요
1위. 0.4933239817619324, 7168, 그 제품이 펄이라서 그래요

2위. 0.45893675088882446, 3419, 다음에 또 들려주세요

3위. 0.45887666940689087, 6784, 네, 저런 제품도 괜찮고 이 제품도 얇게 봄신상으로 나왔어요

4위. 0.4425618052482605, 2570, 네

5위. 0.43961089849472046, 3876, 그럼 이 디자인 어떠세요?

신발이 너무 가벼운거같아요 좀 무게감있는건 없나요?
1위. 0.5984174609184265, 4381, 그럼 이 가방은 어떠세요?

2위. 0.5915377140045166, 2595, 저거보다 작은 걸로요?

3위. 0.5821505784988403, 4871, 어깨끈을 조절 해보세요

4위. 0.5692281723022461, 3886, 그럼 9센티힐 추천해 드릴게요

5위. 0.5543965697288513, 1669, 이거 신어보세요 이게 제일

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
